In [ ]:
%load_ext autoreload
%autoreload 2
import os
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
from train_main import create_dataset
from train_main import pre_process
import utils
from utils import device
from models import *
import constants
from matplotlib.pyplot import savefig
import forecaster


## Batch Scheduler to train

In [ ]:
## !sbatch --gres=gpu:titanxp:1 --mem=32G run.sh

## Test for latest data points

In [ ]:
df = pd.read_csv('data/weatherstats_montreal_daily_inter.csv',index_col=0)
df = pre_process(df[:360], save_plots=True)

In [ ]:
max_v = create_dataset(df[df['Year'] >= 2018], look_back=28, forecast_horizon=15, batch_size=1)

In [ ]:
max_y_plot = []
for input_x, max_y, no_z in max_v:
    max_y_plot.append(max_y[0][0])

In [ ]:
print (len(max_y_plot))
plt.figure(figsize=(16,3.5))
plt.title("Maxima Temprature per Forecasting Horizon")
plt.xlabel("Date/Time")
plt.ylabel("Values")
plt.plot(max_y_plot,c="red",alpha=5,label='maxima temprature')

plt.legend()

In [ ]:
plt.figure(1,figsize=(7,5.5))
plt.subplot(211)
plt.title('Full time series data',fontsize=14)
plt.xlabel("Days",fontsize=14)
plt.ylabel("Values",fontsize=14)
plt.plot(df[['max_temperature']][:360],'r')


plt.subplot(212)
plt.title('Maxima Temprature per Forecasting Horizon',fontsize=14)

plt.xlabel("Forecasting Horizon",fontsize=14)
plt.ylabel("Values",fontsize=14)

plt.plot(max_y_plot[:360])

plt.tight_layout()
from matplotlib.pyplot import savefig
savefig('./plots/data_maxi.eps')

plt.show()


In [ ]:
#ax = df[['max_temperature', 'avg_hourly_temperature','avg_temperature','min_temperature']].plot(title='Full Time Series Data',fontsize=13, figsize = (7,2.5))


## LSTM Forecaster

In [ ]:
ep_loss, mean, rms, lower, upper, mae, acc, test_true_y, test_pred_y = forecaster.LSTMForecaster(df)

In [ ]:
print (rms,mae)

In [ ]:
forecast_fi = {}
fn = 0
for i in range(0, len(ep_loss), 15):
    sp = i
    ep = i + 15
    forecast_fi[fn] = ep_loss[sp:ep]
    fn+=1
    

fig, axs = plt.subplots(nrows=5, ncols=6, figsize=(20,10 ))

for ax, i in zip(axs.flat, forecast_fi):
    ax.plot(forecast_fi[i])
    ax.set_title(str(i)+"\'th Forecasting Cycle Error in oC")

plt.tight_layout()
plt.show()
# forecast_fi

In [ ]:
fig, ax = plt.subplots(figsize=(7,4))
ax.plot(np.array(test_true_y[:100]).reshape(-1),c='blue', label='GT', alpha=5)
ax.plot(mean[:100], label='Prediction', c='green', linestyle='--', alpha=5)
ax.set(title="Maxima Forecasting (LSTM)", ylabel="Max temprature", xlabel="Days") #ylim=(12.7, 13.8))
ax.legend();
savefig("plots/lstm_pred.eps")

In [ ]:
pred_fo = {}
gt_fo = {}
fn = 0
for i in range(0, len(mean), 15):
    sp = i
    ep = i + 15
    pred_fo[fn] = mean[sp:ep]
    fn+=1
fn = 0
for i in range(0, len(test_true_y), 15):
    sp = i
    ep = i + 15
    gt_fo[fn] = test_true_y[sp:ep]
    fn+=1
    

fig, axs = plt.subplots(nrows=5, ncols=6, figsize=(20,10 ))

for ax, i in zip(axs.flat, pred_fo):
    ax.plot(pred_fo[i])
    ax.plot(gt_fo[i])
    ax.set_title(str(i)+"\'th Forecasting Cycle")

plt.tight_layout()
plt.show()
savefig("plots/lstm_forecast_cycle.eps")

## GU-LSTM Forecaster

In [ ]:
ep_loss, mean, rms, lower, upper, mae, acc, test_true_y, test_pred_y = forecaster.LSTMGUForecaster(df)

In [ ]:
print (rms,mae)

In [ ]:
forecast_fi = {}
fn = 0
for i in range(0, len(ep_loss), 15):
    sp = i
    ep = i + 15
    forecast_fi[fn] = ep_loss[sp:ep]
    fn+=1
    

fig, axs = plt.subplots(nrows=5, ncols=6, figsize=(20,10 ))

for ax, i in zip(axs.flat, forecast_fi):
    ax.plot(forecast_fi[i])
    ax.set_title(str(i)+"\'th Forecasting Cycle Error in oC")

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(7,4))
ax.plot(np.array(test_true_y[:100]).reshape(-1),c='blue', label='GT', alpha=5)
ax.plot(mean[:100], label='Prediction', c='green', linestyle='--', alpha=5)
ax.set(title="Maxima Forecasting (GU+LSTM)", ylabel="Max temprature", xlabel="Days") #ylim=(12.7, 13.8))
ax.legend();
savefig("plots/gu_lstm_pred.eps")

In [ ]:
pred_fo = {}
gt_fo = {}
fn = 0
for i in range(0, len(mean), 15):
    sp = i
    ep = i + 15
    pred_fo[fn] = mean[sp:ep]
    fn+=1
fn = 0
for i in range(0, len(test_true_y), 15):
    sp = i
    ep = i + 15
    gt_fo[fn] = test_true_y[sp:ep]
    fn+=1
    

fig, axs = plt.subplots(nrows=5, ncols=6, figsize=(20,10 ))

for ax, i in zip(axs.flat, pred_fo):
    ax.plot(pred_fo[i])
    ax.plot(gt_fo[i])
    ax.set_title(str(i)+"\'th Forecasting Cycle")

plt.tight_layout()
plt.show()
savefig("plots/gu_lstm_forecast_cycle.eps")

## ENCDEC Forecaster

In [ ]:
ep_loss, mean, rms, lower, upper, mae, acc, test_true_y, test_pred_y = forecaster.ENCDECForecaster(df)

In [ ]:
print (mae,rms)

In [ ]:
fig, ax = plt.subplots(figsize=(7,4))
ax.plot(np.array(test_true_y[:100]).reshape(-1),c='blue', label='GT', alpha=5)
ax.plot(mean[:100], label='Prediction', c='green', linestyle='--', alpha=5)
ax.set(title="Maxima Forecasting (ENCDEC LSTM)", ylabel="Max temprature", xlabel="Days") #ylim=(12.7, 13.8))
ax.legend();
savefig("plots/enc_dec.eps")

In [ ]:
pred_fo = {}
gt_fo = {}
fn = 0
for i in range(0, len(mean), 15):
    sp = i
    ep = i + 15
    pred_fo[fn] = mean[sp:ep]
    fn+=1
fn = 0
for i in range(0, len(test_true_y), 15):
    sp = i
    ep = i + 15
    gt_fo[fn] = test_true_y[sp:ep]
    fn+=1
    

fig, axs = plt.subplots(nrows=5, ncols=6, figsize=(20,10 ))

for ax, i in zip(axs.flat, pred_fo):
    ax.plot(pred_fo[i])
    ax.plot(gt_fo[i])
    ax.set_title(str(i)+"\'th Forecasting Cycle")

plt.tight_layout()
savefig("plots/enc_dec_forecast_cycle.eps")
plt.show()